In [1]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import pandas as pd
import os

In [2]:
default_dir = os.getcwd()
print("CURRENT DIR",os.getcwd())
path = './unzip/'
os.chdir(path)
print("UPDATED DIR",os.getcwd())
labels = [name for name in os.listdir('.') if os.path.isdir(name)]
print(f'Total Labels: {len(labels)} \n')
labels

CURRENT DIR E:\INEURON_DATA\Audio-Processing\notebooks
UPDATED DIR E:\INEURON_DATA\Audio-Processing\notebooks\unzip
Total Labels: 2 



['cat', 'dog']

In [3]:

data_path = '../data/spectrograms/' 
dog_cat_dataset = datasets.ImageFolder(
    root=data_path,
    transform=transforms.Compose([transforms.Resize((201,81)),
                                  transforms.ToTensor()
                                  ])
)
print(dog_cat_dataset)

Dataset ImageFolder
    Number of datapoints: 4159
    Root location: ../data/spectrograms/
    StandardTransform
Transform: Compose(
               Resize(size=(201, 81), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )


- `ImageFolder` automatically creates the image class labels and indices based on the folders for each audio class.
- We'll use the `class_to_idx` to view the class mapping for the image dataset.

In [5]:
class_map=dog_cat_dataset.class_to_idx
print("\nClass category and index of the images: {}\n".format(class_map))


Class category and index of the images: {'cat': 0, 'dog': 1}



In [6]:
#split data to test and train
#use 80% to train
train_size = int(0.8 * len(dog_cat_dataset))
test_size = len(dog_cat_dataset) - train_size
dog_cat_train_dataset, dog_cat_test_dataset = torch.utils.data.random_split(dog_cat_dataset, [train_size, test_size])

print("Training size:", len(dog_cat_train_dataset))
print("Testing size:",len(dog_cat_test_dataset))

Training size: 3327
Testing size: 832


Because the dataset was randomly split, let's count the training data to verify that the data has a fairly even distribution between the images in the `dog` and `cat` categories.

In [7]:
from collections import Counter

# labels in training set
train_classes = [label for _, label in dog_cat_train_dataset]
Counter(train_classes)

Counter({1: 1718, 0: 1609})

In [9]:
train_dataloader = torch.utils.data.DataLoader(
    dog_cat_train_dataset,
    batch_size=15,
    num_workers=0,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    dog_cat_test_dataset,
    batch_size=15,
    num_workers=0,
)

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [11]:
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(51136, 50)
        self.fc2 = nn.Linear(50, 2)


    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #x = x.view(x.size(0), -1)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x,dim=1)  

model = CNNet().to(device)

In [12]:
# cost function used to determine best parameters
cost = torch.nn.CrossEntropyLoss()

# used to create optimal parameters
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Create the training function

def train(dataloader, model, loss, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, Y) in enumerate(dataloader):
        
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = cost(pred, Y)
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')


# Create the validation/test function

def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (X, Y) in enumerate(dataloader):
            X, Y = X.to(device), Y.to(device)
            pred = model(X)

            test_loss += cost(pred, Y).item()
            correct += (pred.argmax(1)==Y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size

    print(f'\nTest Error:\nacc: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f}\n')

In [13]:
epochs = 15

for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    train(train_dataloader, model, cost, optimizer)
    test(test_dataloader, model)
print('Done!')

Epoch 1
-------------------------------
loss: 0.704774  [    0/ 3327]
loss: 0.693147  [ 1500/ 3327]
loss: 0.693147  [ 3000/ 3327]

Test Error:
acc: 50.7%, avg loss: 0.046654

Epoch 2
-------------------------------
loss: 0.693147  [    0/ 3327]
loss: 0.693147  [ 1500/ 3327]
loss: 0.693147  [ 3000/ 3327]

Test Error:
acc: 50.7%, avg loss: 0.046654

Epoch 3
-------------------------------
loss: 0.693147  [    0/ 3327]
loss: 0.693147  [ 1500/ 3327]
loss: 0.693147  [ 3000/ 3327]

Test Error:
acc: 50.7%, avg loss: 0.046654

Epoch 4
-------------------------------
loss: 0.693147  [    0/ 3327]
loss: 0.693147  [ 1500/ 3327]
loss: 0.693147  [ 3000/ 3327]

Test Error:
acc: 50.7%, avg loss: 0.046654

Epoch 5
-------------------------------
loss: 0.693147  [    0/ 3327]
loss: 0.693147  [ 1500/ 3327]
loss: 0.693147  [ 3000/ 3327]

Test Error:
acc: 50.7%, avg loss: 0.046654

Epoch 6
-------------------------------
loss: 0.693147  [    0/ 3327]
loss: 0.693147  [ 1500/ 3327]
loss: 0.693147  [ 3000/ 3

## Predictions

In [ ]:
model.eval()
test_loss, correct = 0, 0
class_map = ['no', 'yes']

with torch.no_grad():
    for batch, (X, Y) in enumerate(test_dataloader):
        X, Y = X.to(device), Y.to(device)
        pred = model(X)
        print("Predicted:\nvalue={}, class_name= {}\n".format(pred[0].argmax(0),class_map[pred[0].argmax(0)]))
        print("Actual:\nvalue={}, class_name= {}\n".format(Y[0],class_map[Y[0]]))
        